In [61]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
import json

In [62]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
#pc.create_index(
 #   name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
#)

In [63]:

data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Hawthorne',
  'subject': 'English Literature',
  'stars': 4,
  'review': 'Engaging lectures and thought-provoking assignments. Sometimes a bit disorganized.'},
 {'professor': 'Prof. Alan Turing',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Brilliant mind, explains complex concepts clearly. Challenging but rewarding course.'},
 {'professor': 'Dr. Marie Curie',
  'subject': 'Physics',
  'stars': 5,
  'review': 'Inspiring teacher with a passion for the subject. Experiments were fascinating.'},
 {'professor': 'Prof. John Nash',
  'subject': 'Mathematics',
  'stars': 3,
  'review': 'Brilliant mathematician, but lectures can be hard to follow. Office hours very helpful.'},
 {'professor': 'Dr. Jane Goodall',
  'subject': 'Anthropology',
  'stars': 5,
  'review': 'Incredible field experience shared. Makes primatology accessible and exciting.'},
 {'professor': 'Prof. Stephen Hawking',
  'subject': 'Astrophysics',
  'stars': 4,
  'review': 'Mind-bending 

In [64]:
process_data=[]
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    process_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [65]:
process_data[0]

{'values': [-0.026572159,
  0.00061695813,
  0.0016452216,
  0.024480898,
  0.0267184,
  0.0063432436,
  0.005612034,
  -0.017607529,
  0.006003231,
  0.03729169,
  0.027040131,
  0.021819295,
  -0.0029321506,
  -0.0042007994,
  0.018850584,
  -0.0010803622,
  -0.022448136,
  -0.022316517,
  0.011070514,
  0.023150098,
  0.020620111,
  -0.02208253,
  0.03319692,
  0.020166762,
  -0.05057046,
  -0.04039202,
  -0.018587349,
  0.015106791,
  0.006482173,
  0.05019023,
  0.06346899,
  -0.027873712,
  0.013688244,
  -0.031061785,
  -0.046943657,
  0.09096248,
  0.023588823,
  0.038110647,
  0.03509806,
  0.038490877,
  0.027683597,
  0.033401657,
  -0.026733024,
  -0.034893323,
  0.02102959,
  -0.018031629,
  -0.022404263,
  -0.044311304,
  0.05954971,
  0.032524206,
  -0.03132502,
  0.031529758,
  0.05598141,
  0.0006475775,
  -0.014989797,
  -0.040655255,
  -0.026937762,
  0.031792995,
  -0.002124164,
  0.008460095,
  0.036004763,
  -0.013169086,
  0.051594153,
  0.018324113,
  -0.0156625

In [66]:
index = pc.Index('rag')
index.upsert(
    vectors=process_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [67]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}